In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from preprocessy.pipelines import StandardPipeline

In [3]:
def test_input():
    params = {'target_label':"Survived",'cat_cols':['Pclass','Sex','Parch','Embarked'],'drop_cols':["PassengerId","Name","Ticket","Cabin"],"fill_missing":[["Age"],"mean"]}
    p = StandardPipeline(train_df_path="/Users/mac/pypi/preprocessy/datasets/titanic.csv",test_df_path="/Users/mac/pypi/preprocessy/datasets/titanic.csv", params=params)
    p.save_config('/Users/mac/pypi/preprocessy/preprocessy/config.json')
    p.process()
    
    return params
    
    

In [4]:
params = test_input()
params[train_df]


Pipeline Class: StandardPipeline

+-----------------------+--------------------------------------------------+
| Pipeline Property     | Value                                            |
+-----------------------+--------------------------------------------------+
| Train Dataframe Path  | /Users/mac/pypi/preprocessy/datasets/titanic.csv |
| Test Dataframe Path   | /Users/mac/pypi/preprocessy/datasets/titanic.csv |
| Config File Path      | None                                             |
| Pipeline Stages       | Read file, Execute                               |
| Total Pipeline Stages | 2                                                |
| Total Params          | 6                                                |
+-----------------------+--------------------------------------------------+

Processing...

==> Completed Stage: Read file

==> Completed Stage: Execute

Pipeline Stages |████████████████████████████████████████| 2/2 [100%] in 0.3s (6

Pipeline Completed Successfully



NameError: name 'train_df' is not defined

In [6]:
params['train_df']

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.000000,1,0,7.2500,S
1,1,1,female,38.000000,1,0,71.2833,C
2,1,3,female,26.000000,0,0,7.9250,S
3,1,1,female,35.000000,1,0,53.1000,S
4,0,3,male,35.000000,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.000000,0,0,13.0000,S
887,1,1,female,19.000000,0,0,30.0000,S
888,0,3,female,29.642093,1,2,23.4500,S
889,1,1,male,26.000000,0,0,30.0000,C


In [23]:
train_new = params["X_train"]
test_new = params["X_test"]
train_y = train_new["Survived"]
test_y = test_new["Survived"]
colstodrop = ["Survived","Sex","Parch","SurvivedEncoded","PclassEncoded","Embarked"]
train_new.drop(colstodrop,axis=1,inplace=True)
test_new.drop(colstodrop,axis=1,inplace=True)

In [12]:

import pandas as pd
from sklearn.model_selection import train_test_split
original = pd.read_csv("/Users/mac/preprocessy/datasets/titanic.csv")
cols = ["PassengerId","Name","Ticket","Cabin"]
original.drop(cols,axis=1,inplace=True)
original.dropna(inplace=True)
original["SexEncoded"] = original["Sex"].apply(lambda x: 0 if x=='female' else 1)
original["EmbarkedEncoded"] = original["Embarked"].apply(lambda x: 0 if x=='C' else( 1 if x=='S' else 2))
original.drop(['Sex','Embarked'],axis=1,inplace=True)
y = original["Survived"]
X = original.drop(["Survived"],axis=1)
trainx, testx, trainy, testy = train_test_split(X,y,test_size=0.2)


In [8]:
from sklearn.linear_model import LogisticRegression


In [24]:
l = LogisticRegression(max_iter=200)
l.fit(train_new,train_y)

LogisticRegression(max_iter=200)

In [26]:
print("model score: %.3f" % l.score(test_new, test_y))

model score: 0.817


In [20]:
def sc(X,y):
    trainx, testx, trainy, testy = train_test_split(X,y,test_size=0.2)
    l = LogisticRegression(max_iter=200)
    l.fit(trainx,trainy)
    t = l.score(testx, testy)
    return t

In [21]:
z = []
for _ in range(5):
    z.append(sc(X,y))
sum(z)/len(z)


0.793006993006993

In [19]:
z

[0.8391608391608392,
 0.8391608391608392,
 0.8391608391608392,
 0.8391608391608392,
 0.8391608391608392]